## Using `cz-benchmarks`

You may duplicate this notebook and replace the simulated model execution cell with your own model code.

This notebook guides you through loading single-cell datasets, running your model, and evaluating results using standardized tasks and metrics.

All you need to do is swap in your model’s output—no extra setup required.
Use the provided examples as templates for your workflow.

In [ ]:
# Setup you notebook kernel and install the package
# Install czbenchmarks for the selected Jupyter kernel
!pip install czbenchmarks

### 1. Datasets

Datasets are wrapped for consistent loading and compatibility:

- `SingleCellLabeledDataset`: Gene expression data with cell labels (supports clustering, embedding, label prediction).
- `SingleCellPerturbationDataset`: Perturbation datasets with control and perturbed cells.

In [9]:

import numpy as np
from czbenchmarks.datasets import load_dataset
from czbenchmarks.datasets.single_cell_labeled import SingleCellLabeledDataset

#### List Available Datasets 

This code snippet lists all available datasets in the czbenchmarks library.

In [ ]:

from czbenchmarks.datasets.utils import list_available_datasets
import pandas as pd

# List all available datasets in czbenchmarks
available_datasets = list_available_datasets()

# Display available datasets as a table
df_datasets = pd.DataFrame({'Dataset': available_datasets})
df_datasets

,Dataset
0,adamson_perturb
1,chicken_spermatogenesis
2,chimpanzee_spermatogenesis
3,dixit_perturb
4,gorilla_spermatogenesis
5,human_spermatogenesis
6,marmoset_spermatogenesis
7,mouse_spermatogenesis
8,norman_perturb
9,opossum_spermatogenesis


####  Load a Dataset

Load the pre-configured `tsv2_prostate dataset`, which you can find in the list above. The library will automatically download, cache, and load this dataset as a SingleCellLabeledDataset object. This makes it easy to reuse the data for your analysis without extra setup.

**Loaded dataset provides:**
- `dataset.adata`: AnnData object with gene expression data.
- `dataset.labels`: pandas Series of cell type labels.

In [11]:
# The 'dataset' object is a validated AnnData wrapper, ensuring efficient downstream processing.
dataset: SingleCellLabeledDataset = load_dataset("tsv2_prostate")

INFO:czbenchmarks.file_utils:File already exists in cache: /Users/sgupta/.cz-benchmarks/datasets/homo_sapiens_10df7690-6d10-4029-a47e-0f071bb2df83_Prostate_v2_curated.h5ad
INFO:czbenchmarks.datasets.single_cell:Loading dataset from /Users/sgupta/.cz-benchmarks/datasets/homo_sapiens_10df7690-6d10-4029-a47e-0f071bb2df83_Prostate_v2_curated.h5ad


### 2. Model

Tasks expect a `CellRepresentation`, which is a `numpy.ndarray` with cells as rows and embedding features as columns. For demonstration, we simulate model output with random data.

For this example, we will use random numbers to simulate what a real model would produce. In your own work, you should replace this with the actual output from your model—such as the embeddings generated by your neural network or other method.

---

> **Tip**: You can copy this notebook and swap out the code below for your own model's import, inference, or training steps. Just make sure the final output is a NumPy array in the correct shape.


In [12]:
# Simulated 10-dimensional embedding for each cell
# Replace this with your model's actual code to generate output embeddings for tasks like clustering, embedding, or label prediction.
from czbenchmarks.tasks.types import CellRepresentation
model_output: CellRepresentation = np.random.rand(dataset.adata.shape[0], 10)


### 3. Task

Each task defines an evaluation workflow with `run()` and `compute_baseline()` methods.

| Task Name         | Class                         | Purpose                                         |
|-------------------|-------------------------------|-------------------------------------------------|
| Clustering        | `ClusteringTask`              | Evaluate cell group separation                  |
| Embedding Quality | `EmbeddingTask`               | Assess embedding structure                      |
| Label Prediction  | `MetadataLabelPredictionTask` | Predict labels from embeddings                  |
| Batch Integration | `BatchIntegrationTask`        | Evaluate batch integration                      |
| Cross-Species     | `CrossSpeciesIntegrationTask` | Integrate data across species                   |

#### Task Metrics

Metrics are managed by `MetricRegistry` and returned as `MetricResult` objects.

- `MetricType`: Enum of metric names (e.g., `ADJUSTED_RAND_INDEX`, `SILHOUETTE_SCORE`)
- `MetricResult`: Stores metric type, value, and parameters

All tasks compute and return metrics automatically.

#### Example: Run a Clustering Task

Evaluate the embedding by measuring clustering performance using Adjusted Rand Index (ARI) and Normalized Mutual Information (NMI). The task compares Leiden clusters from the embedding to true labels. Higher scores indicate better clustering.

Compare `clustering_results` to `clustering_baseline_results` to assess model performance against the PCA baseline.

In [13]:
from czbenchmarks.tasks import (
    ClusteringTask,
)
from czbenchmarks.tasks.clustering import ClusteringTaskInput

# Evaluate the embedding by measuring clustering performance using Adjusted Rand Index (ARI) and Normalized Mutual Information (NMI). The task compares Leiden clusters from the embedding to true labels. Higher scores indicate better clustering. Compare `clustering_results` to `clustering_baseline_results` to assess model performance against the PCA baseline.

# 1. Initialize the task
clustering_task = ClusteringTask()

# 2. Define the inputs for the task
clustering_task_input = ClusteringTaskInput(
    obs=dataset.adata.obs,  # The full observation metadata
    input_labels=dataset.labels,  # The ground-truth labels for comparison
)

# 3. Run the task on your model's output
clustering_results = clustering_task.run(
    cell_representation=model_output,
    task_input=clustering_task_input,
)

# 4. Compute and run the baseline for comparison
expression_data = dataset.adata.X
clustering_baseline = clustering_task.compute_baseline(expression_data)
clustering_baseline_results = clustering_task.run(
    cell_representation=clustering_baseline,
    task_input=clustering_task_input,
)

print("--- Clustering Model Results ---")
for result in clustering_results:
    print(result.model_dump_json(indent=2))

print("\n--- Clustering Baseline Results ---")
for result in clustering_baseline_results:
    print(result.model_dump_json(indent=2))

--- Clustering Model Results ---
{
  "metric_type": "adjusted_rand_index",
  "value": -0.0019522836814914062,
  "params": {}
}
{
  "metric_type": "normalized_mutual_info",
  "value": 0.01784275562902829,
  "params": {}
}

--- Clustering Baseline Results ---
{
  "metric_type": "adjusted_rand_index",
  "value": 0.626707020983652,
  "params": {}
}
{
  "metric_type": "normalized_mutual_info",
  "value": 0.8326481406592264,
  "params": {}
}
